In [1]:
import glob
from keras.models import load_model
import mat4py
import pandas as pd
import cv2
import numpy as np
import os

In [3]:
path = 'C:/Users/brian/Desktop/AI_intro/final_class/'
#inform = mat4py.loadmat(path+'devkit/devkit/cars_test_annos_withlabels.mat')
#df = pd.DataFrame(inform['annotations'])

df = pd.DataFrame()
filetype = ['*jpeg', '*jpg', '*png']

cases=[]
for f in filetype:
    cases.extend(glob.glob(path+'classifier_evaluation/'+f))

cases.sort()
df['path'] = cases
name = []
for case in cases:
    name.append(os.path.basename(case))
df['name'] = name
df.head()

,path,name
0,C:/Users/brian/Desktop/AI_intro/final_class/cl...,1.jpg
1,C:/Users/brian/Desktop/AI_intro/final_class/cl...,10.jpg
2,C:/Users/brian/Desktop/AI_intro/final_class/cl...,100.jpg
3,C:/Users/brian/Desktop/AI_intro/final_class/cl...,1000.jpg
4,C:/Users/brian/Desktop/AI_intro/final_class/cl...,1001.jpg


In [4]:
#data augmentation
def adjust_data(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_NEAREST)
    img = np.reshape(img, (224,224,3))
    return img

def compose_data(df):
    data = []
    
    for ind in df.index:
        path = df['path'][ind]
        
        data.append(adjust_data(path))
        
    return np.array(data)

In [5]:
fitting = compose_data(df)

In [32]:
cv2.imshow('image', fitting[1])
cv2.waitKey(0)

-1

In [6]:
MobileNet = load_model("C:/Users/brian/Desktop/AI_intro/final_class/Mobile")

In [7]:
predict=[]
for x in fitting:
    x = np.expand_dims(x, axis=0)
    prob = MobileNet.predict(x)
    predict_class = prob.argmax(axis=-1)
    predict.append(predict_class[0])

In [8]:
FinalOutput = pd.DataFrame()
FinalOutput['filename'] = df['name']
FinalOutput['label'] = predict

FinalOutput.head()

,filename,label
0,1.jpg,118
1,10.jpg,86
2,100.jpg,46
3,1000.jpg,104
4,1001.jpg,94


In [9]:
FinalOutput.to_csv('Group2.CSV', index=False)